In [1]:
from google.colab import drive 
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
import pandas as pd
# cd /content/drive/MyDrive/PL

In [3]:
cd /content/drive/MyDrive/PL

/content/drive/MyDrive/PL


In [17]:
data = pd.read_csv('/content/drive/MyDrive/PL/meta_data_public_test.csv')
print((data['Wav_Path'][0].split("/")[7]).split(".")[0] + ".npy")

37333433375f3638.npy


In [19]:
# /content/drive/MyDrive/output_alignment_fusion/3130303538355f3338.npy
!pip install torch
# from torch.utils.data import Dataset
# import matplotlib.pyplot as plt
# import os
import pandas as pd
import torch.nn.functional as F
import torch
import numpy as np
# import numpy as np
# from char_embedding import tensor_to_text
import matplotlib.pyplot as plt
import glob
base_dir = '/content/drive/MyDrive/song_phonetic/'
# pat = glob.glob('/content/drive/MyDrive/output_alignment/*.npy')
last_output = []
PA = []
for i in range(len(data)):
  tu = str(base_dir) + data['Wav_Path'][i].split("/")[7].split(".")[0] + ".npy"
  PA.append(tu)
  x = np.load(tu)

  # print(x)

  x = torch.tensor(x)
  predicted_ids = torch.argmax(torch.tensor(x), dim=1)


  x = torch.log_softmax(x, dim=-1)
  x = x.cpu().detach()


  labels = ('ắ', 'ồ', 'z', 'ứ', 'ỡ', 'ì', 'x', 'ặ', 'u', 'ẹ', 'd', 'ỵ', 'r', 'p', 't', 'ỳ', 'ẩ', 'f', 'ó', 'á', 'v', 'ã', 'i', 'ư', 'ở', 'ễ', 'ụ', 'ú', 'ũ', ' ', 'ă', 'é', 'ằ', 'a', 'ấ', 'ờ', 'ữ', 'ớ', 'n', 'ý', 's', 'h', 'ơ', 'ị', 'l', 'c', 'k', 'ỷ', 'ỗ', 'ế', 'ẻ', 'ợ', 'ẫ', 'í', 'ỏ', 'ủ', 'g', 'q', 'j', 'ò', 'ỹ', 'ự', 'ô', 'b', 'y', 'ĩ', 'ỉ', 'ẵ', 'ầ', 'ê', 'ộ', 'ậ', 'm', 'ń', 'o', 'ọ', 'đ', 'ẽ', 'ử', 'à', 'è', 'e', 'ẳ', 'ổ', 'ù', 'w', 'ả', 'ạ', 'â', 'ệ', 'ề', 'õ', 'ố', 'ể', 'ừ',)

  def clean_corpus(str1):
      res1 = ""
      for i in str1:
          if i.isalpha() or i==" ":
              res1 = "".join([res1, i])
      return res1.lower()
  transcript = " " + str(data['Lyric_Path'][i]) + ""
  transcript = clean_corpus(transcript)
  dictionary = {c: i for i, c in enumerate(labels)}

  tokens = [dictionary[c] for c in transcript]


  def get_trellis(x, tokens, blank_id=95):
      num_frame = x.size(0)
      num_tokens = len(tokens)

      # Trellis has extra diemsions for both time axis and tokens.
      # The extra dim for tokens represents <SoS> (start-of-sentence)
      # The extra dim for time axis is for simplification of the code.
      trellis = torch.empty((num_frame + 1, num_tokens + 1))
      trellis[0, 0] = 0
      trellis[1:, 0] = torch.cumsum(x[:, 0], 0)
      trellis[0, -num_tokens:] = -float("inf")
      trellis[-num_tokens:, 0] = float("inf")

      for t in range(num_frame):
          trellis[t + 1, 1:] = torch.maximum(
              # Score for staying at the same token
              trellis[t, 1:] + x[t, blank_id],
              # Score for changing to the next token
              trellis[t, :-1] + x[t, tokens],
          )
      return trellis


  trellis = get_trellis(x, tokens)

  from dataclasses import dataclass
  @dataclass
  class Point:
    token_index: int
    time_index: int
    score: float


  def backtrack(trellis, emission, tokens, blank_id=95):
    # Note:
    # j and t are indices for trellis, which has extra dimensions 
    # for time and tokens at the beginning.
    # When refering to time frame index `T` in trellis,
    # the corresponding index in emission is `T-1`.
    # Similarly, when refering to token index `J` in trellis,
    # the corresponding index in transcript is `J-1`.
    j = trellis.size(1) - 1
    t_start = torch.argmax(trellis[:, j]).item()

    path = []
    for t in range(t_start, 0, -1):
      # 1. Figure out if the current position was stay or change
      # Note (again):
      # `emission[J-1]` is the emission at time frame `J` of trellis dimension.
      # Score for token staying the same from time frame J-1 to T.
      stayed = trellis[t-1, j] + emission[t-1, blank_id]
      # Score for token changing from C-1 at T-1 to J at T.
      changed = trellis[t-1, j-1] + emission[t-1, tokens[j-1]]

      # 2. Store the path with frame-wise probability.
      prob = emission[t-1, tokens[j-1] if changed > stayed else 0].exp().item()
      # Return token index and time index in non-trellis coordinate.
      path.append(Point(j-1, t-1, prob))

      # 3. Update the token
      if changed > stayed:
        j -= 1
        if j == 0:
          break
    else:
      raise ValueError('Failed to align')
    return path[::-1]

  path = backtrack(trellis, x, tokens)
  @dataclass
  class Segment:
      label: str
      start: int
      end: int
      score: float
      

      def __repr__(self):
        return f"{self.label}\t({self.score:4.2f}): [{self.start:5d}, {self.end:5d})"

      @property
      def length(self):
          return self.end - self.start


  def merge_repeats(path):
      i1, i2 = 0, 0
      segments = []
      while i1 < len(path):
          while i2 < len(path) and path[i1].token_index == path[i2].token_index:
              i2 += 1
          score = sum(path[k].score for k in range(i1, i2)) / (i2 - i1)
          segments.append(
              Segment(
                  transcript[path[i1].token_index],
                  path[i1].time_index,
                  path[i2 - 1].time_index + 1,
                  score,
              )
          )
          i1 = i2
      return segments


  segments = merge_repeats(path)

  # Merge words
  def merge_words(segments, separator=" "):
      words = []
      i1, i2 = 0, 0
      while i1 < len(segments):
          if i2 >= len(segments) or segments[i2].label == separator:
              if i1 != i2:
                  segs = segments[i1:i2]
                  word = "".join([seg.label for seg in segs])
                  score = sum(seg.score * seg.length for seg in segs) / sum(seg.length for seg in segs)
                  words.append(Segment(word, segments[i1].start*20, segments[i2 - 1].end*20, score))
              i1 = i2 + 1
              i2 = i1
          else:
              i2 += 1
      return words


  word_segments = merge_words(segments)
  last_output.append(word_segments)
data = pd.DataFrame([PA,last_output])
data = data.transpose() #To Transpose and make each rows as columns
data.columns=['Path','Output']
data.to_csv('/content/drive/MyDrive/PL/song_output.csv')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<ipython-input-19-448198f8689f>:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted_ids = torch.argmax(torch.tensor(x), dim=1)


In [15]:
data.to_csv('/content/drive/MyDrive/PL/vocal_output.csv')